# Map Layers

In [ ]:
import os
import time

from felt_python import (
    create_map,
    delete_map,
    get_map,
    list_layers,
    upload_file,
    upload_url,
    upload_dataframe,
    upload_geodataframe,
    refresh_file_layer,
    refresh_url_layer,
    get_layer,
    update_layer_style,
    get_export_link,
    download_layer,
    update_layers,
    delete_layer,
    create_custom_export,
    get_custom_export_status
)

os.environ["FELT_API_TOKEN"] = "<YOUR_API_TOKEN>"

# Maps

Create a map and retrieve its details, storing the ID in a variable

In [ ]:
resp = create_map(
    title="A felt-python map for testing layers",
    lat=40,
    lon=-3,
    zoom=10,
    public_access="private",
)
map_id = resp["id"]

In [ ]:
get_map(map_id)

# Layers

Create layers from file, URL and (Geo)DataFrame uploads

## Upload a file

In [ ]:
metadata = {
    "attribution_text": "Sample Data",
    "source_name": "Felt Python Library",
    "description": "Sample points near Null Island"
}

layer_resp = upload_file(
    map_id=map_id, 
    file_name="fixtures/null-island-points-sample.geojson", 
    layer_name="The Points Layer",
    metadata=metadata
)
layer_id = layer_resp["layer_id"]

### Refresh file upload


In [ ]:
# Wait for layer processing
while get_layer(map_id, layer_id)["progress"] < 100:
    print("Waiting for layer to finish processing...")
    time.sleep(5)refresh_file_layer(map_id, layer_id, file_name="fixtures/null-island-points.geojson")

## Upload a URL

In [ ]:
live_earthquakes_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"
url_upload = upload_url(map_id, live_earthquakes_url, "Live Earthquakes")
url_layer_id = url_upload["layer_id"]

### Refresh URL upload

Wait for upload to finish first

In [ ]:
while get_layer(map_id, url_layer_id)["progress"] < 100:
    print("Waiting for layer to finish processing...")
    time.sleep(5)
refresh_url_layer(map_id, url_layer_id)

# Styling

Fetch style of file upload layer

In [ ]:
current_style = get_layer(map_id, layer_id)["style"]
current_style

Update style

In [ ]:
new_style = current_style.copy()
new_style["color"] = "red"
new_style["size"] = 20
update_layer_style(map_id, layer_id, new_style)

# Update multiple layers

You can update multiple layers at once with a single API call

In [ ]:
# Get the first two layers
layers = list_layers(map_id)[:2]

# Prepare updates for both layers
layer_updates = [
    {
        "id": layers[0]["id"],
        "name": "Updated Layer 1",
        "caption": "New caption for layer 1"
    },
    {
        "id": layers[1]["id"],
        "name": "Updated Layer 2",
        "caption": "New caption for layer 2"
    }
]

# Update both layers at once
updated_layers = update_layers(map_id, layer_updates)

# DataFrames and GeoDataFrames

In [ ]:
import pandas as pd

df = pd.read_csv("fixtures/null-island-polygons-wkt.csv")
upload_dataframe(map_id, df, "Polygons from a CSV")

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('fixtures/null-island-points.geojson')
upload_geodataframe(map_id, gdf, layer_name="GeoPandas Upload")

## List all uploaded layers

In [ ]:
list_layers(map_id)

# Custom layer exports

Create a custom export with filters

In [ ]:
# Custom export with filters
export_request = create_custom_export(
    map_id=map_id,
    layer_id=layer_id,
    output_format="csv",  # Options: "csv", "gpkg", "geojson"
    filters=[{"attribute": "name", "operator": "=", "value": "Sample Point"}],
    email_on_completion=True
)

print(f"Export request ID: {export_request['export_request_id']}")
print(f"Polling endpoint: {export_request['poll_endpoint']}")

# Poll for export status
export_id = export_request['export_request_id']

while True:
    export_status = get_custom_export_status(
        map_id=map_id,
        layer_id=layer_id,
        export_id=export_id
    )
    
    print(f"Export status: {export_status['status']}")
    
    if export_status['status'] == 'completed':
        print(f"Download URL: {export_status['download_url']}")
        break
    elif export_status['status'] == 'failed':
        print("Export failed")
        break
        
    time.sleep(5)

# Downloading a layer

Download the file uploaded layer as a GeoPackage

In [ ]:
# Get just the export link
export_link = get_export_link(map_id, layer_id)
print(f"Export link: {export_link}")

# Download the layer to a file
output_file = download_layer(map_id, layer_id, file_name="exported.gpkg")
print(f"Downloaded to: {output_file}")

# Delete a layer

In [ ]:
# Get a list of all layers
all_layers = list_layers(map_id)
print(f"Number of layers before deletion: {len(all_layers)}")

# Delete the URL layer
delete_layer(map_id, url_layer_id)

# Verify deletion
all_layers = list_layers(map_id)
print(f"Number of layers after deletion: {len(all_layers)}")

# Cleanup

In [ ]:
delete_map(map_id)